In [ ]:
from rotalysis.pump.pump_function import PumpFunction as pf
import pandas as pd
from rotalysis import definitions as defs
from pathlib import Path

In [ ]:
rated_flow = 100
rated_head = 1000
rated_speed = 3000
rated_efficeincy = 0.7

In [ ]:
pf.get_pump_efficiency(rated_flow, rated_efficeincy, 80)

In [ ]:
df = pd.DataFrame(
    columns=[
        defs.ComputedVariables.FLOWRATE_PERCENT,
        defs.ComputedVariables.OLD_PUMP_EFFICIENCY,
    ],
    data=[
        {
            defs.ComputedVariables.FLOWRATE_PERCENT: n,
            defs.ComputedVariables.OLD_PUMP_EFFICIENCY: 0,
        }
        for n in range(30, 101, 10)
    ],
)
df

In [ ]:
df[defs.PumpOperationVariables.DISCHARGE_FLOWRATE] = (
    df[defs.ComputedVariables.FLOWRATE_PERCENT] * rated_flow / 100
)
df

In [ ]:
df[defs.ComputedVariables.OLD_PUMP_EFFICIENCY] = df.apply(
    lambda x: pf.get_pump_efficiency(
        rated_flow, rated_efficeincy, x[defs.PumpOperationVariables.DISCHARGE_FLOWRATE]
    ),
    axis=1,
)

In [ ]:
df

In [ ]:
str(Path("data/input.xlsx").resolve())

In [ ]:
input_xl = pd.ExcelFile(str(Path("data/input.xlsx").resolve()))
pump_curve = pd.read_excel(
    input_xl,
    sheet_name=defs.InputSheetNames.PUMP_CURVE,
    usecols="E:F",
    header=2,
)
pump_curve.dropna(inplace=True)
pump_curve.rename(columns={"flow.1": "flow", "head.1": "head"}, inplace=True)
pump_curve

In [ ]:
from plotly import graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=pump_curve["flow"],
        y=pump_curve["head"],
        mode="lines",
        name="Pump Curve",
    )
)
fig.update_layout(
    title="Pump Curve",
    xaxis=dict(
        title="Flowrate",
        range=[0, 900],  # Set the upper and lower range for the x-axis
    ),
    yaxis=dict(title=dict(text="Head"), range=[0, 900]),
)
fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Given data
flow_rate = 500  # m³/h
head_at_flow_rate = 50  # m
head_noflow = 20  # m

# Convert flow rate from m³/h to m³/s for calculation consistency in some contexts
flow_rate_m3_s = flow_rate / 3600

# Calculate k using the operating point
k = (head_at_flow_rate - head_noflow) / (flow_rate_m3_s**2)

# Generate flow rates from 0 to 600 m³/h for plotting
flow_rates = np.linspace(0, 600, 100)  # m³/h
flow_rates_m3_s = flow_rates / 3600  # Convert to m³/s for calculation

# Calculate the heads for each flow rate
heads = head_noflow + k * (flow_rates_m3_s**2)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(flow_rates, heads, label="System Curve", color="blue")
plt.scatter(
    flow_rate, head_at_flow_rate, color="red", label="Operating Point (500 m³/h, 50 m)"
)
plt.title("Pump System Curve")
plt.xlabel("Flow Rate (m³/h)")
plt.ylabel("Head (m)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def calculate_system_curve(flow_rate_m3_h, head_at_flow_rate, static_head):
    """
    Calculate the pump system curve equation from a given operating point and static head.

    Parameters:
    - flow_rate_m3_h: The flow rate at the operating point in cubic meters per hour (m³/h).
    - head_at_flow_rate: The head at the operating point in meters (m).
    - static_head: The static head in meters (m).

    Returns:
    - k: The constant for the system curve equation.
    - system_curve_eq: A function that calculates the head (H) for any given flow rate in m³/h.
    """
    # Convert flow rate from m³/h to m³/s for calculation consistency
    flow_rate_m3_s = flow_rate_m3_h / 3600

    # Calculate k using the operating point
    k = (head_at_flow_rate - static_head) / (flow_rate_m3_s**2)

    # Define the system curve equation
    def system_curve_eq(flow_rate_m3_h):
        flow_rate_m3_s = flow_rate_m3_h / 3600
        return static_head + k * (flow_rate_m3_s**2)

    return k, system_curve_eq


# Example usage
k, quad_curve = calculate_system_curve(500, 50, 20)

# Print the system curve equation for a range of flow rates
flow_rates = [0, 100, 200, 300, 400, 500, 600]
head_values = [quad_curve(q) for q in flow_rates]
for q, h in zip(flow_rates, head_values):
    print(f"Flow Rate: {q} m³/h, Head: {h:.2f} m")

SymPy Trail

In [ ]:
from sympy import symbols, Eq, solve, init_printing, Function, plot

In [ ]:
# Define the flow rate as a symbol
Q, H = symbols("Q H")
a, b, c = (0.01, 0, 3)
expr = a * Q**2 + b * Q + c
sytem_curve_eq = Eq(expr, H)
plot(expr, (Q, 0, 600))

In [ ]:
from sympy import symbols, Eq, solve, plot, evalf

# Define the symbols and constants
Q, H = symbols("Q H")
a = symbols("a")
b, c = (0, 3)

# Define the expression
expr = a * Q**2 + b * Q + c
system_curve_eq = Eq(expr, H)

# Plotting the equation from Q = 0 to Q = 600 (using an example value for a since a is not specified)
example_a = 1  # example coefficient for a
plot_expr = expr.subs(a, example_a)
p = plot(
    plot_expr,
    (Q, 0, 600),
    show=False,
    title=f"(System Curve for {expr})",
    ylabel="H",
    xlabel="Q",
)
p.show()

# Solve the equation for a given H=2000 and Q=500
knowns = {Q: 500, H: 2000}
solve_for_a = solve(system_curve_eq.subs(knowns), a)

In [ ]:
import numpy as np
from scipy.optimize import fsolve
from plotly import graph_objects as go

In [ ]:
from scipy.optimize import fsolve


# Define the system curve function
def quad_curve(flow, head, static_head, a, b):
    return (a * flow**2) + (b * flow) + static_head - head


# Function for fsolve
def equation_to_solve(a):
    return quad_curve(500, 2000, 20, a, 10)  # Plugging in the given values


# Solve for a using fsolve with an initial guess
a_solution = fsolve(equation_to_solve, x0=0)  # Initial guess for a
print(a_solution)

In [ ]:
def equation_to_solve(b):
    return quad_curve(500, 2000, 20, -0.01208, b)


b_solution = fsolve(equation_to_solve, x0=0)
b_solution

In [ ]:
def quad_curve(flow, head, head_noflow, a, b=0):
    return (a * flow**2) + (b * flow) + head_noflow - head

In [ ]:
rated_flow, rated_head, shutoff_head = (100, 1000, 1300)

In [ ]:
def pump_curve(a):
    return quad_curve(rated_flow, rated_head, shutoff_head, a, 0)

In [ ]:
a_solution = fsolve(pump_curve, x0=0)
a_solution

In [ ]:
def pump_curve_eq(flow, shutoff_head, a, b=0):
    return (a * flow**2) + (b * flow) + shutoff_head

In [ ]:
h = pump_curve_eq([100, 200], 1300, -0.03)
h


In [36]:
import numpy as np
from scipy.optimize import fsolve, curve_fit
from plotly import graph_objects as go

In [37]:
def head_curve(flow, a, b=0, noflow_head=0):
    if isinstance(flow, (int, float)):
        return (a * flow**2) + (b * flow) + noflow_head
    if isinstance(flow, (list, np.ndarray)):
        return [(a * q**2) + (b * q) + noflow_head for q in flow]

In [38]:
def solve_coefficient(flow, head, initial_guess, b=0, constant=0):
    equation = lambda a: head_curve(flow, a, b, constant) - head

    solution = fsolve(equation, x0=initial_guess)
    return solution[0]

In [39]:
rated_flow, rated_head, shutoff_head = (100, 1000, 1300)
a = solve_coefficient(rated_flow, rated_head, 0, 0, shutoff_head)

In [40]:
head = head_curve(flow=[100, 200], a=a, noflow_head=shutoff_head)
head

[1000.0, 100.00000000000023]

In [45]:
# Example data points
Q = np.array([0, 100, 200, 300, 400, 500])  # Flow rates
H = np.array([1300, 1200, 1000, 700, 300, 100])  # Corresponding heads

In [47]:
eq = lambda Q, a, b: a * Q**2 + b * Q + 1300
curve_fit(eq, Q, H)

(array([-0.00267081, -1.18012422]),
 array([[ 7.39988822e-07, -3.02722696e-04],
        [-3.02722696e-04,  1.31718007e-01]]))

In [49]:
def get_headcurve_coefficients(flow, head):
    if flow[0] == 0:
        c = head[0]
        eq = lambda Q, a, b: (a * Q**2) + (b * Q) + c
        popt, _ = curve_fit(eq, flow, head)
        return (popt[0], popt[1], c)
    else:
        eq = lambda Q, a, b, c: (a * Q**2) + (b * Q) + c
        popt, _ = curve_fit(eq, flow, head)
        return tuple(popt)

In [52]:
# Example data points
Q = np.array([100, 200, 300, 400, 500])  # Flow rates
H = np.array([1200, 1000, 700, 300, 100])  # Corresponding heads
get_headcurve_coefficients(Q, H)

(-0.0007142857164672556, -2.4714285714368445, 1480.0000000033526)

In [50]:
get_headcurve_coefficients(Q, H)

(-0.0026708074632431173, -1.1801242180491813, 1300)